### 分组测试额外方法
2022-07-22
- 重新生成数据并保存
2022-08-10
- 原始特征需要加入volume

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import importlib
import sys
sys.path.append('C:/Users/Administrator/Desktop/Repositories/Low-Frequency-Spread-Estimator')
sys.path.append('C:/Users/18316/Desktop/Repositories/Low-Frequency-Spread-Estimator')
sys.path.append('C:/Users/Handsome Bad Guy/Desktop/Repositories/Low-Frequency-Spread-Estimator')

from SpreadEstimator.SpreadEstimator import SpreadEstimator
from mytools.AutoTester import AutoTester

from time import time

import warnings
warnings.filterwarnings("ignore")

se = SpreadEstimator()

In [2]:
univ = se.data.data_dic['volume'] > 0  # 当天有交易
th = 0.8

univ_2 = []
for i in range(2081):
    if np.sum(se.data.data_dic['close'][univ[:,i],i]>5) >= th * 243:
        univ_2.append(1)
    else:
        univ_2.append(0)
univ_2 = np.array(univ_2) > 0
univ_2 = np.repeat(univ_2.reshape(1,2081), 243, 0)

In [3]:
def test_cs(signal, target, univ, back: int = 1):
    corr = []
    for i in range(signal.shape[0]):
        if i < back-1:
            continue
        t = np.nanmean(target[i-back+1: i+1], axis=0)
        se = (~np.isnan(t)) & (~np.isnan(signal[i])) & univ[i]
        if np.sum(se) < 2:
            continue
        corr.append(np.corrcoef(signal[i, se], t[se])[0, 1])
    print('{:.4f}'.format(np.mean(corr)))
    return corr

def test_ts(signal, target, univ, back: int = 1):
    corr = []
    new_t = np.zeros((len(target)-back+1, target.shape[1]))
    for i in range(new_t.shape[0]):
        new_t[i] = np.nanmean(target[i:i+back])
    for i in range(signal.shape[1]):
        t = new_t[:, i]
        se = (~np.isnan(t)) & (~np.isnan(signal[back-1:, i])) & univ[back-1:, i]
        if np.sum(se) < 2:
            continue
        corr.append(np.corrcoef(signal[back-1:, i][se], t[se])[0, 1])
    print('{:.4f}'.format(np.mean(corr)))
    return corr

In [4]:
def test_rela(signal, all_univ):
    print('rela cs')
    for u in all_univ:
        _ = test_cs(signal, se.data.spread_dic['relative_spread'], u, 20)
    print()

    print('rela ts')
    for u in all_univ:
        _ = test_ts(signal, se.data.spread_dic['relative_spread'], u, 20)
    print()

    print('vol_wtd_rel_bas cs ')
    for u in all_univ:
        _ = test_cs(signal, se.data.spread_dic['vol_wtd_rel_bas'], u, 20)
    print()

    print('vol_wtd_rel_bas ts ')
    for u in all_univ:
        _ = test_ts(signal, se.data.spread_dic['vol_wtd_rel_bas'], u, 20)
        
def test_spread(signal, all_univ):
    print('spread cs')
    for u in all_univ:
        _ = test_cs(signal, se.data.spread_dic['spread'], u, 20)
    print()

    print('spread ts')
    for u in all_univ:
        _ = test_ts(signal, se.data.spread_dic['spread'], u, 20)

### 以下测试额外方法并保存signal数据

In [7]:
signal_rela = np.zeros((243, 2081, 12), dtype=np.float32)  # 将LOT和Gibbs也加入
signal_spread = np.zeros((243, 2081, 12), dtype=np.float32)

In [9]:
# 价格和成交量以及市值
names = ['open', 'close', 'high', 'low']
for i in range(len(names)):
    name = names[i]
    fml = 'tsmean{logv{' + name + '},20}'
    stats, signal = se.test_factor(fml, corr_type='linear', method='cs', spread_type='relative_spread')
    test_rela(signal, [univ_2])
    signal_rela[:, :, 6+i] = signal
    
    fml = 'tsmean{' + name + ',20}'
    stats, signal = se.test_factor(fml, corr_type='linear', method='cs', spread_type='relative_spread')
    test_rela(signal, [univ_2])
    signal_spread[:, :, 6+i] = signal
    
names = ['volume', 'cap']
for i in range(len(names)):
    name = names[i]
    fml = 'tsmean{logv{' + name + '},20}'
    stats, signal = se.test_factor(fml, corr_type='linear', method='cs', spread_type='relative_spread')
    test_rela(signal, [univ_2])
    signal_rela[:, :, 10+i] = signal
    signal_spread[:, :, 10+i] = signal

mean corr: -0.7744, positive_corr_ratio: 0.0000, corr_IR: -23.9731
rela cs
-0.6461

rela ts
-0.3886

vol_wtd_rel_bas cs 
-0.6783

vol_wtd_rel_bas ts 
-0.3743
mean corr: -0.3925, positive_corr_ratio: 0.0000, corr_IR: -9.9491
rela cs
-0.3624

rela ts
-0.3807

vol_wtd_rel_bas cs 
-0.3883

vol_wtd_rel_bas ts 
-0.3662
mean corr: -0.7741, positive_corr_ratio: 0.0000, corr_IR: -23.8753
rela cs
-0.6454

rela ts
-0.3879

vol_wtd_rel_bas cs 
-0.6775

vol_wtd_rel_bas ts 
-0.3720
mean corr: -0.3920, positive_corr_ratio: 0.0000, corr_IR: -9.9100
rela cs
-0.3616

rela ts
-0.3797

vol_wtd_rel_bas cs 
-0.3874

vol_wtd_rel_bas ts 
-0.3637
mean corr: -0.7743, positive_corr_ratio: 0.0000, corr_IR: -23.9354
rela cs
-0.6456

rela ts
-0.3759

vol_wtd_rel_bas cs 
-0.6770

vol_wtd_rel_bas ts 
-0.3590
mean corr: -0.3920, positive_corr_ratio: 0.0000, corr_IR: -9.9942
rela cs
-0.3615

rela ts
-0.3678

vol_wtd_rel_bas cs 
-0.3870

vol_wtd_rel_bas ts 
-0.3507
mean corr: -0.7743, positive_corr_ratio: 0.0000, corr_I

### 保存经典方法

In [10]:
# HL-rela
beta = 'prod{tsmean{powv{minus{logv{high},logv{low}},2},2},2}'

high_1 = 'logv{tsdelay{high,1}}'
low_1 = 'logv{tsdelay{low,1}}'
con_1 = 'condition{gt{logv{low},logv{tsdelay{close,1}}},minus{logv{low},logv{tsdelay{close,1}}},minus{close,close}}'
con_2 = 'condition{lt{logv{high},logv{tsdelay{close,1}}},minus{logv{high},logv{tsdelay{close,1}}},minus{close,close}}'
con = 'add{' + con_1 + ',' + con_2 + '}'
high_2 = 'minus{logv{high},' + con + '}'
low_2 = 'minus{logv{low},' + con + '}'
high = 'condition{ge{' + high_1 + ',' + high_2 + '},' + high_1 + ',' + high_2 + '}'
low = 'condition{le{' + low_1 + ',' + low_2 + '},' + low_1 + ',' + low_2 + '}'

gamma = 'powv{minus{' + high + ',' + low + '},2}'
alpha = 'div{prod{' + 'powv{' + beta + ',0.5},0.4142},0.1716}'
fml = 'minus{' + alpha + ',' + 'powv{' + 'div{' + gamma + ',0.1716},0.5}}'

fml = 'div{' + 'minus{expv{' + fml + '},1},' + 'add{expv{' + fml + '},1}}'
fml = 'condition{' + 'ge{' + fml + ',0},' + fml + ',add{minus{close,close},0}}'
fml = 'tsmean{' + fml + ',20}'

fml = 'div{' + 'minus{expv{' + fml + '},1},' + 'add{expv{' + fml + '},1}}'
fml = 'condition{' + 'ge{' + fml + ',0},' + fml + ',add{minus{close,close},0}}'
fml = 'tsmean{' + fml + ',20}'
stats, signal = se.test_factor(fml, corr_type='linear', method='cs', spread_type='relative_spread')
signal_rela[:, :, 1] = signal

mean corr: -0.2082, positive_corr_ratio: 0.0000, corr_IR: -3.6871


In [11]:
# Roll-rela
a = 'tsdelta{logv{close},1}'
b = 'tsdelay{tsdelta{logv{close},1},1}'
fml = 'prod{' + a + ',' + b + '}'
fml = 'condition{' + 'ge{' + fml + ',0},' + fml + ',add{minus{close,close},0}}'
fml = 'tsmean{' + fml + ',20}'
fml = 'powv{' + fml + ',0.5}'

stats, signal = se.test_factor(fml, corr_type='linear', method='cs', spread_type='spread')
signal_rela[:, :, 0] = signal

mean corr: 0.0994, positive_corr_ratio: 0.9279, corr_IR: 1.2409


In [12]:
# CHL-rela
a = 'minus{logv{tsdelay{close,1}},div{add{logv{tsdelay{high,1}},logv{tsdelay{low,1}}},2}}'
b = 'minus{logv{tsdelay{close,1}},div{add{logv{high},logv{low}},2}}'
fml = 'prod{' + a + ',' + b + '}'
fml = 'condition{' + 'ge{' + fml + ',0},' + fml + ',add{minus{close,close},0}}'
fml = 'tsmean{' + fml + ',20}'
fml = 'powv{' + fml + ',0.5}'

stats, signal = se.test_factor(fml, corr_type='linear', method='cs', spread_type='relative_spread')
signal_rela[:, :, 5] = signal

mean corr: -0.2317, positive_corr_ratio: 0.0000, corr_IR: -3.9265


In [15]:
# HL-spread
beta = 'prod{tsmean{powv{minus{high,low},2},2},2}'
high_1 = 'tsdelay{high,1}'
low_1 = 'tsdelay{low,1}'
con_1 = 'condition{gt{low,tsdelay{close,1}},minus{low,tsdelay{close,1}},minus{close,close}}'
con_2 = 'condition{lt{high,tsdelay{close,1}},minus{high,tsdelay{close,1}},minus{close,close}}'
con = 'add{' + con_1 + ',' + con_2 + '}'
high_2 = 'minus{high,' + con + '}'
low_2 = 'minus{low,' + con + '}'
high = 'condition{ge{' + high_1 + ',' + high_2 + '},' + high_1 + ',' + high_2 + '}'
low = 'condition{le{' + low_1 + ',' + low_2 + '},' + low_1 + ',' + low_2 + '}'

gamma = 'powv{minus{' + high + ',' + low + '},2}'
alpha = 'div{prod{' + 'powv{' + beta + ',0.5},0.4142},0.1716}'
fml = 'minus{' + alpha + ',' + 'powv{' + 'div{' + gamma + ',0.1716},0.5}}'

fml = 'div{' + 'minus{expv{' + fml + '},1},' + 'add{expv{' + fml + '},1}}'
fml = 'condition{' + 'ge{' + fml + ',0},' + fml + ',add{minus{close,close},0}}'
fml = 'tsmean{' + fml + ',20}'
stats, signal = se.test_factor(fml, corr_type='linear', method='cs', spread_type='relative_spread')

signal_spread[:, :, 1] = signal

mean corr: -0.4630, positive_corr_ratio: 0.0000, corr_IR: -16.7947


In [16]:
a = 'tsdelta{close,1}'
b = 'tsdelay{tsdelta{close,1},1}'
fml = 'prod{' + a + ',' + b + '}'
fml = 'condition{' + 'ge{' + fml + ',0},' + fml + ',add{minus{close,close},0}}'
fml = 'tsmean{' + fml + ',20}'
fml = 'powv{' + fml + ',0.5}'

stats, signal = se.test_factor(fml, corr_type='linear', method='cs', spread_type='spread')

signal_spread[:, :, 0] = signal

mean corr: 0.7924, positive_corr_ratio: 1.0000, corr_IR: 18.7505


In [17]:
a = 'minus{tsdelay{close,1},div{add{tsdelay{high,1},tsdelay{low,1}},2}}'
b = 'minus{tsdelay{close,1},div{add{high,low},2}}'
fml = 'prod{' + a + ',' + b + '}'
fml = 'condition{' + 'ge{' + fml + ',0},' + fml + ',add{minus{close,close},0}}'
fml = 'tsmean{' + fml + ',20}'
fml = 'powv{' + fml + ',0.5}'

stats, signal = se.test_factor(fml, corr_type='linear', method='cs', spread_type='relative_spread')
signal_spread[:, :, 5] = signal

mean corr: -0.3688, positive_corr_ratio: 0.0000, corr_IR: -8.7364


#### LOT和Gibbs

In [20]:
import pickle
with open(r"D:\Documents\学习资料\本科毕业论文\data\LOT.pkl", 'rb') as f:
    LOT = pickle.load(f)
signal_rela[:, :,  2:4] = LOT
signal_spread[:, :, 2:4] = LOT

In [21]:
path = r"C:\Users\18316\Desktop\Repositories\Low-Frequency-Spread-Estimator\estimator\cache"
gibbs = np.zeros((243, 2081), dtype=np.float32)
for i in range(9):
    gibbs[20+21*i:41+21*i] = np.load('{}/{}.npy'.format(path, i))
i = 9
gibbs[20+21*i:] = np.load('{}/{}.npy'.format(path, i))

signal_rela[:, :,  4] = gibbs
signal_spread[:, :, 4] = gibbs

In [22]:
import pickle
with open(r"D:\Documents\学习资料\本科毕业论文\data\signal_rela.pkl", 'wb') as f:
    pickle.dump(signal_rela, f)
with open(r"D:\Documents\学习资料\本科毕业论文\data\signal_spread.pkl", 'wb') as f:
    pickle.dump(signal_spread, f)

### 测试新的算子

In [204]:
fml = 'tsmean{absv{div{minus{close,open},minus{high,close}}},20}'
stats, signal = se.test_factor(fml, corr_type='linear', method='cs', spread_type='relative_spread')
print()

test_rela(signal, [univ_2])

mean corr: -0.2177, positive_corr_ratio: 0.0000, corr_IR: -4.5677

rela cs
-0.0957

rela ts
0.1054

vol_wtd_rel_bas cs 
-0.1091

vol_wtd_rel_bas ts 
0.1255


In [ ]:
fmls = [
    'tsmean{div{close,open},20}',
    'tsmean{absv{tspct{close,1}},20}',
    'tsmean{logv{volume},20}',
    'tsmean{tspct{minus{div{high,low},1},1},19}',
    'tsstd{minus{div{high,low},1},20}',
    'tsmean{absv{div{minus{close,open},minus{high,close}}},20}'
]

### 尝试将四个对数价格放进去，看看能不能学到

In [9]:
signal_rela = np.zeros((243, 2081, 4 + 4))

In [12]:
# 四个价格
fmls = [
    'tsmean{logv{open},20}',
    'tsmean{logv{close},20}',
    'tsmean{logv{high},20}',
    'tsmean{logv{low},20}',
    'tsmean{logv{cap},20}',
]

for i in range(len(fmls)):
    fml = fmls[i]

    stats, signal = se.test_factor(fml, corr_type='linear', method='cs', spread_type='relative_spread')
    # signal[signal!=0] = 1 / signal[signal!=0]
    print()

    test_rela(signal, [univ_2])
    # print()
    # signal[signal!=0] = 1 / signal[signal!=0]
    # test_spread(signal, [univ_2])

    signal_rela[:, :, i] = signal

mean corr: -0.7744, positive_corr_ratio: 0.0000, corr_IR: -23.9731

rela cs
-0.2840

rela ts
-0.3795

vol_wtd_rel_bas cs 
-0.3280

vol_wtd_rel_bas ts 
-0.3675
mean corr: -0.7741, positive_corr_ratio: 0.0000, corr_IR: -23.8753

rela cs
-0.2830

rela ts
-0.3788

vol_wtd_rel_bas cs 
-0.3267

vol_wtd_rel_bas ts 
-0.3654
mean corr: -0.7743, positive_corr_ratio: 0.0000, corr_IR: -23.9354

rela cs
-0.2828

rela ts
-0.3689

vol_wtd_rel_bas cs 
-0.3256

vol_wtd_rel_bas ts 
-0.3545
mean corr: -0.7743, positive_corr_ratio: 0.0000, corr_IR: -23.8779

rela cs
-0.2843

rela ts
-0.3894

vol_wtd_rel_bas cs 
-0.3291

vol_wtd_rel_bas ts 
-0.3783
mean corr: -0.4011, positive_corr_ratio: 0.0000, corr_IR: -5.4286

rela cs
-0.5754

rela ts
-0.5267

vol_wtd_rel_bas cs 
-0.5782

vol_wtd_rel_bas ts 
-0.5170


### renew target

In [25]:
target_wrela = se.data.spread_dic['relative_spread'].copy()
for i in range(20,243):
    t = np.nanmean(se.data.spread_dic['vol_wtd_rel_bas'][i-20:i],axis=0)
    target_wrela[i-1] = t

In [23]:
se.data.spread_dic.keys()

dict_keys(['spread', 'relative_spread', 'vol_wtd_rel_bas', 'vol_wtd_bas'])

In [214]:
with open(r"D:\Documents\学习资料\本科毕业论文\data\target_rela.pkl", 'wb') as f:
    pickle.dump(target_rela, f)

In [26]:
with open(r"D:\Documents\学习资料\本科毕业论文\data\target_wrela.pkl", 'wb') as f:
    pickle.dump(target_wrela, f)

In [219]:
import torch
x = torch.zeros(2, dtype=torch.int)
y = torch.clone(x)

In [220]:
torch.nanmean(x)

RuntimeError: nanmean(): expected input to have floating point dtype but got Int

In [222]:
torch.where(x==0)

(tensor([0, 1]),)